original: [openai-cookbook/examples/How_to_finetune_chat_models.ipynb at main · openai/openai-cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb)

# How to fine-tune chat models

このノートブックは、新しいgpt-3.5-turboのファインチューニングのためのガイドを提供します。

[RecipeNLG](https://github.com/Glorf/recipenlg)データセットを使用し、エンティティ抽出を実行します。

このデータセットは、様々なレシピと、それぞれから抽出された一般的な成分のリストを提供します。これは、固有表現認識（NER）タスクに共通のデータセットです。

次の手順を実行します。

1. セットアップ：データセットをロードし、ファインチューニングする1つのドメインに絞り込みます。
2. データの準備：トレーニングと検証のサンプルを作成し、それらをファイルエンドポイントにアップロードすることで、ファインチューニング用のデータを準備します。
3. ファインチューニング：ファインチューニングモデルを作成します。
4. 推論：新しい入力の推論に微調整されたモデルを使用します。

これが終わるまでに、ファインチューニングされたgpt-3.5-turboモデルをトレーニング、評価、デプロイできるようになります。

ファインチューニングの詳細：

- [Fine-tuning - OpenAI API](https://platform.openai.com/docs/guides/fine-tuning)
- [API Reference - OpenAI API](https://platform.openai.com/docs/api-reference/fine-tuning)
- [GPT-3.5 Turbo fine-tuning and API updates](https://openai.com/blog/gpt-3-5-turbo-fine-tuning-and-api-updates)

## Setup

In [1]:
# openai pythonパッケージの最新バージョンを必ず使用する
%pip install -U openai

     ---------------------------------------- 75.5/75.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.27.8
    Uninstalling openai-0.27.8:
      Successfully uninstalled openai-0.27.8
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import openai
import os
import pandas as pd
from pprint import pprint

ファインチューニングは、特定のドメインに焦点を当てた場合に最適に機能します。データセットがモデルが学習できるように十分に焦点が絞られていると同時に、まだ見たことのないサンプルが見逃されない程度に一般的であることを確認することが重要です。

これを念頭に置いて、 www.cookbooks.com のドキュメントのみを含むようにRecipeNLGデータセットからサブセットを抽出しました。

In [3]:
# このタスクに使用するデータセットを読み込みます
# これはRecipeNLGデータセットになります。 www.cookbooks.comのドキュメントのみを含むようにクリーンアップしました。
recipe_df = pd.read_csv("./data/cookbook_recipes_nlg_10k.csv")
recipe_df.head()

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


## データの準備

データを準備するところから始めます。

ChatCompletion形式でファインチューニングする場合、各トレーニングサンプルはメッセージの単純なリストになります。たとえば、エントリは次のようになります。

```
[{'role': 'system',
  'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},
```

```
 {'role': 'user',
  'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},
```

```
 {'role': 'assistant',
  'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]
```

トレーニングプロセス中、この会話は分割され、最後のエントリはモデルが生成する完了であり、残りのメッセージはプロンプトとして機能します。

トレーニングサンプルを作成するときは、これを考慮してください。モデルが複数ターンの会話で動作する場合は、会話が拡大し始めたときにパフォーマンスが低下しないように、代表的なサンプルを提供してください。

現在、各トレーニングサンプルには4096トークンの制限があることに注意してください。これより長いものは4096トークンで切り捨てられます。

In [4]:
training_data = []

system_message = "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = f"""Title: {row['title']}\n\nIngredients: {row['ingredients']}\n\nGeneric ingredients: """
    messages.append({"role": "user", "content": user_message})

    messages.append({"role": "assistant", "content": row["NER"]})

    return {"message": messages}

pprint(prepare_example_conversation(recipe_df.iloc[0]))


{'message': [{'content': 'You are a helpful recipe assistant. You are to '
                         'extract the generic ingredients from each of the '
                         'recipes provided.',
              'role': 'system'},
             {'content': 'Title: No-Bake Nut Cookies\n'
                         '\n'
                         'Ingredients: ["1 c. firmly packed brown sugar", "1/2 '
                         'c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. '
                         'broken nuts (pecans)", "2 Tbsp. butter or '
                         'margarine", "3 1/2 c. bite size shredded rice '
                         'biscuits"]\n'
                         '\n'
                         'Generic ingredients: ',
              'role': 'user'},
             {'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", '
                         '"bite size shredded rice biscuits"]',
              'role': 'assistant'}]}


次に、トレーニングデータとして使用するデータセットのサブセットに対してこれを実行してみましょう。

30~50この適切に剪定された例から始めることもできます。トレーニングセットのサイズを増やすと、パフォーマンスが直線的に拡大し続けることがわかりますが、ジョブの時間も長くなります。

In [5]:
# データセットの最初の100行をトレーニングに使用します
training_df = recipe_df.loc[0:100]

# prepare_example_conversation関数をtraining_dfの各行に適用します
training_data = training_df.apply(prepare_example_conversation, axis=1).to_list()

for example in training_data[:5]:
    print(example)

{'message': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]}
{'message': [{'role': 'system', 'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'}, {'role': 'user', 'content': 'Title: Jewell Ball\'S Chicken\n\nIngredients: ["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"]\n\nGeneric ingredients: '}, {'role': 'assistant', 'content': '["beef"

トレーニングデータに加えて、モデルがトレーニングセットに過剰適合していないことを確認するために使用される検証データもオプションで提供できます。

In [6]:
validation_df = recipe_df.loc[101:200]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

次に、データを`.jsonl`ファイルとして保存する必要があります。各行は1つのトレーニング会話例です。

In [7]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [8]:
training_file_name = "tmp_recipe_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_recipe_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

トレーニング`.jsonl`ファイルの最初の5行は次のようになります。

In [13]:
# トレーニングファイルの最初の5行を出力します
# for linux
# !head -n 5 tmp_recipe_finetune_training.jsonl
# for windows (powershell)
!Powershell.exe -Command "type .\tmp_recipe_finetune_training.jsonl -Head 5"

{"message": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "Title: No-Bake Nut Cookies\n\nIngredients: [\"1 c. firmly packed brown sugar\", \"1/2 c. evaporated milk\", \"1/2 tsp. vanilla\", \"1/2 c. broken nuts (pecans)\", \"2 Tbsp. butter or margarine\", \"3 1/2 c. bite size shredded rice biscuits\"]\n\nGeneric ingredients: "}, {"role": "assistant", "content": "[\"brown sugar\", \"milk\", \"vanilla\", \"nuts\", \"butter\", \"bite size shredded rice biscuits\"]"}]}
{"message": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "Title: Jewell Ball'S Chicken\n\nIngredients: [\"1 small jar chipped beef, cut up\", \"4 boned chicken breasts\", \"1 can cream of mushroom soup\", \"1 carton sour cream\"]\n\nGeneric ingredients: "}, {"role": "

# 参考情報

- [固有表現抽出（NER）とはどんなタスクか【自然言語処理タスク紹介1】 | もじとばコム](https://mojitoba.com/2021/03/03/what_is_ner/)
- [【python】jsonっぽいファイル「jsonl（JSON LINES）」をcsvに書き換えるまで｜じゅにこ](https://note.com/junico02_se_blog/n/n5ed9c474aed4)
- [RecipeNLG](https://recipenlg.cs.put.poznan.pl/)
